#  Workshop Initiation

In [ ]:
region = 'us-east-1'
model_id = "anthropic.claude-3-sonnet-20240229-v1:0"
mistral7b = "mistral.mistral-7b-instruct-v0:2"
mistrasmall = "mistral.mistral-small-2402-v1:0"
model_arn = f'arn:aws:bedrock:{region}::foundation-model/{model_id}'
session_id = None

%pip install -U "boto3" "botocore" "langchain-community"

import json
import boto3
from helpers import *


bedrock_runtime = boto3.client('bedrock-runtime')
bedrock_agent_runtime = boto3.client('bedrock-agent-runtime')

In [ ]:
def invoke_mistral_model(prompt, modelId):
    request_body = json.dumps({
        "prompt": "<s>[INST] " + prompt + " [/INST]",
        "max_tokens" : 200, 
        "temperature" : 0, 
        "top_p" : 0.9, 
        "top_k" : 50
    })
    response = bedrock_runtime.invoke_model(
            body = request_body,
            modelId = modelId
    )
    response_body = json.loads(response.get("body").read())
    return response_body.get('outputs')[0].get('text')

In [ ]:
 def invoke_anthropic_model(modelId, prompt):
        inferenceConfig = { 
            "maxTokens": 400,
            "temperature": 0
        }
        messages = [{
            "role": "user",
            "content": [{
                "text": prompt
            }]
        }]
        response = bedrock_runtime.converse(
            messages = messages,
            inferenceConfig = inferenceConfig,
            modelId = modelId
        )
        return response['output']['message']['content'][0]['text']
modelId = "anthropic.claude-3-haiku-20240307-v1:0"

In [ ]:
region = 'us-east-1'
model_id = "anthropic.claude-3-sonnet-20240229-v1:0"
model_arn = f'arn:aws:bedrock:{region}::foundation-model/{model_id}'
session_id = None

In [ ]:
file_name = "Schema-Rag"
def retrieveAndGenerate(input_text):
    input = {'text': input_text}
    config = {
        'type': 'EXTERNAL_SOURCES',
        'externalSourcesConfiguration': {
            'modelArn': model_arn,
            'sources': [
                {
                    "byteContent": {
                        "contentType": "text/plain",
                        "data": open(file_name, "rb").read(),
                        "identifier": file_name
                      },
                    "sourceType": "BYTE_CONTENT"  
                }
            ]
        }
    }
    response = bedrock_agent_runtime.retrieve_and_generate(
        input = input,
        retrieveAndGenerateConfiguration = config,
        )
    
    return response['output']['text']

# DB Creation

In [ ]:
import pandas as pd
from langchain_community.utilities import SQLDatabase
from sqlalchemy import create_engine

def prepSQLDB():
    df_orders = pd.read_csv("orders.csv", delimiter=',')
    df_offices = pd.read_csv("offices.csv", delimiter=',')
    df_payments = pd.read_csv("payments.csv", delimiter=',')   
    df_products = pd.read_csv("products.csv", delimiter=',')
    df_productlines = pd.read_csv("productlines.csv", delimiter=',')  
    df_customers = pd.read_csv("customers.csv", delimiter=',')
    df_employees = pd.read_csv("employees.csv", delimiter=',')
    df_orderdetails = pd.read_csv("orderdetails.csv", delimiter=',')

    engine = create_engine("sqlite:///anycompany.db")

    df_orders.to_sql("orders", engine, index=False)
    df_offices.to_sql("offices", engine, index=False)
    df_payments.to_sql("payments", engine, index=False)
    df_products.to_sql("products", engine, index=False)
    df_productlines.to_sql("productlines", engine, index=False)
    df_customers.to_sql("customers", engine, index=False)
    df_employees.to_sql("employees", engine, index=False)
    df_orderdetails.to_sql("orderdetails", engine, index=False)
    

In [ ]:
def querySQL(statement):
    engine = create_engine("sqlite:///anycompany.db")
    db = SQLDatabase(engine=engine)
    return db.run(statement)


In [ ]:
prepSQLDB()

# DB Creation Chaos

In [ ]:
import pandas as pd
from langchain_community.utilities import SQLDatabase
from sqlalchemy import create_engine

def prepChaosSQLDB():
    df_orders = pd.read_csv("chaos_orders.csv", delimiter=',')
    df_offices = pd.read_csv("chaos_offices.csv", delimiter=',')
    df_payments = pd.read_csv("chaos_payments.csv", delimiter=',')   
    df_products = pd.read_csv("chaos_products.csv", delimiter=',')
    df_productlines = pd.read_csv("chaos_productlines.csv", delimiter=',')  
    df_customers = pd.read_csv("chaos_customers.csv", delimiter=',')
    df_employees = pd.read_csv("chaos_employees.csv", delimiter=',')
    df_orderdetails = pd.read_csv("chaos_orderdetails.csv", delimiter=',')

    engine = create_engine("sqlite:///chaos_anycompany.db")

    df_orders.to_sql("chaos_orders", engine, index=False)
    df_offices.to_sql("chaos_offices", engine, index=False)
    df_payments.to_sql("chaos_payments", engine, index=False)
    df_products.to_sql("chaos_products", engine, index=False)
    df_productlines.to_sql("chaos_productlines", engine, index=False)
    df_customers.to_sql("chaos_customers", engine, index=False)
    df_employees.to_sql("chaos_employees", engine, index=False)
    df_orderdetails.to_sql("chaos_orderdetails", engine, index=False)
    

In [ ]:
def querySQL_chaos(statement):
    engine = create_engine("sqlite:///chaos_anycompany.db")
    db = SQLDatabase(engine=engine)
    return db.run(statement)


In [ ]:
prepChaosSQLDB()

# Text2SQL 
  


# Scenario  1 - Basic Schema Incorporation
The following scenario showcases how the model can generate an incorrect SQL query due to incorrectly inferring the column names. However, you'll see how providing the schema description in the prompt enables the model to produce the correct query. This activity highlights the importance of contextual information for accurate query generation

  - Model: Mistral AI - Mistral 7B Instruct
  - Techniques:
    - Schema Incorporation
    - Prompt Optimization
  - Use-Case: List all SF employees
    - query1: Without Schema Description
    - query2: With Schema Description
    - query3: With Optimized Schema Description


In [ ]:
question = "List the full names of all San Francisco employees"

prompt = """
You task is to generate the simplest and most effective SQL query. Output only SQL, without comments.
Given tables called: productlines, products, offices, employees, customers, orders, payments, orderdetails


Write a SQL statement to answer the question:
""" + question + """

```sql
"""

In [ ]:
query_1_scenario_1 = invoke_mistral_model(prompt, mistral7b)
print(query_1_scenario_1)

In [ ]:
question = "List the full names of all San Francisco employees"

prompt = """
You task is to generate the simplest and most effective SQL query. Output only SQL, without comments.
Given these tables:
productlines (
  productLine varchar(50),
  textDescription varchar(4000) DEFAULT NULL,
  htmlDescription mediumtext,
  image mediumblob,
  PRIMARY KEY (productLine)
);

products (
  productCode varchar(15),
  productName varchar(70) NOT NULL,
  productLine varchar(50) NOT NULL,
  productScale varchar(10) NOT NULL,
  productVendor varchar(50) NOT NULL,
  productDescription text NOT NULL,
  quantityInStock smallint(6) NOT NULL,
  buyPrice decimal(10,2) NOT NULL,
  MSRP decimal(10,2) NOT NULL,
  PRIMARY KEY (productCode),
  FOREIGN KEY (productLine) REFERENCES productlines (productLine)
);

offices (
  officeCode varchar(10),
  city varchar(50) NOT NULL,
  phone varchar(50) NOT NULL,
  addressLine1 varchar(50) NOT NULL,
  addressLine2 varchar(50) DEFAULT NULL,
  state varchar(50) DEFAULT NULL,
  country varchar(50) NOT NULL,
  postalCode varchar(15) NOT NULL,
  territory varchar(10) NOT NULL,
  PRIMARY KEY (officeCode)
);

employees (
  employeeNumber int,
  lastName varchar(50) NOT NULL,
  firstName varchar(50) NOT NULL,
  extension varchar(10) NOT NULL,
  email varchar(100) NOT NULL,
  officeCode varchar(10) NOT NULL,
  reportsTo int DEFAULT NULL,
  jobTitle varchar(50) NOT NULL,
  PRIMARY KEY (employeeNumber),
  FOREIGN KEY (reportsTo) REFERENCES employees (employeeNumber),
  FOREIGN KEY (officeCode) REFERENCES offices (officeCode)
);

customers (
  customerNumber int,
  customerName varchar(50) NOT NULL,
  contactLastName varchar(50) NOT NULL,
  contactFirstName varchar(50) NOT NULL,
  phone varchar(50) NOT NULL,
  addressLine1 varchar(50) NOT NULL,
  addressLine2 varchar(50) DEFAULT NULL,
  city varchar(50) NOT NULL,
  state varchar(50) DEFAULT NULL,
  postalCode varchar(15) DEFAULT NULL,
  country varchar(50) NOT NULL,
  salesRepEmployeeNumber int DEFAULT NULL,
  creditLimit decimal(10,2) DEFAULT NULL,
  PRIMARY KEY (customerNumber),
  FOREIGN KEY (salesRepEmployeeNumber) REFERENCES employees (employeeNumber)
);

payments (
  customerNumber int,
  checkNumber varchar(50) NOT NULL,
  paymentDate date NOT NULL,
  amount decimal(10,2) NOT NULL,
  PRIMARY KEY (customerNumber,checkNumber),
  FOREIGN KEY (customerNumber) REFERENCES customers (customerNumber)
);

orders (
  orderNumber int,
  orderDate date NOT NULL,
  requiredDate date NOT NULL,
  shippedDate date DEFAULT NULL,
  status varchar(15) NOT NULL,
  comments text,
  customerNumber int NOT NULL,
  PRIMARY KEY (orderNumber),
  FOREIGN KEY (customerNumber) REFERENCES customers (customerNumber)
);

orderdetails (
  orderNumber int,
  productCode varchar(15) NOT NULL,
  quantityOrdered int NOT NULL,
  priceEach decimal(10,2) NOT NULL,
  orderLineNumber smallint(6) NOT NULL,
  PRIMARY KEY (orderNumber,productCode),
  FOREIGN KEY (orderNumber) REFERENCES orders (orderNumber),
  FOREIGN KEY (productCode) REFERENCES products (productCode)
);

Write a SQL statement to answer the question:
""" + question + """

```sql
"""

In [ ]:
query_2_scenario_1 = invoke_mistral_model(prompt, mistral7b)
print(query_2_scenario_1)

In [ ]:
question = "List the full names of all San Francisco employees"

prompt = """
You task is to generate the simplest and most effective SQL query. Output only SQL, without comments.
Given these tables:
offices (
  officeCode varchar(10),
  city varchar(50) NOT NULL,
  phone varchar(50) NOT NULL,
  addressLine1 varchar(50) NOT NULL,
  addressLine2 varchar(50) DEFAULT NULL,
  state varchar(50) DEFAULT NULL,
  country varchar(50) NOT NULL,
  postalCode varchar(15) NOT NULL,
  territory varchar(10) NOT NULL,
  PRIMARY KEY (officeCode)
);

employees (
  employeeNumber int,
  lastName varchar(50) NOT NULL,
  firstName varchar(50) NOT NULL,
  extension varchar(10) NOT NULL,
  email varchar(100) NOT NULL,
  officeCode varchar(10) NOT NULL,
  reportsTo int DEFAULT NULL,
  jobTitle varchar(50) NOT NULL,
  PRIMARY KEY (employeeNumber),
  FOREIGN KEY (reportsTo) REFERENCES employees (employeeNumber),
  FOREIGN KEY (officeCode) REFERENCES offices (officeCode)
);

Write a SQL statement to answer the question:
""" + question + """

```sql
"""

In [ ]:
query_3_scenario_1 = invoke_mistral_model(prompt, mistral7b)
print(query_3_scenario_1)

# Queries Execution



In [ ]:
res = querySQL(query_1_scenario_1)
prettyJSON(res)

In [ ]:
res = querySQL(query_2_scenario_1)
prettyJSON(res)

In [ ]:
res = querySQL(query_3_scenario_1)
prettyJSON(res)

## Results
- Result 1: Error - wrong column name
- Result 2: Works
- Result 3: Works

## Conclusion
- Incoporating the schema within the prompt helps the model to generate a more accurate query
- Optimizing the prompt by reducing irrelevant tables and columns for our question

## Scenario  2 - 
In the following scenario, you will see that simply having the table and column names may not be enough for the model to generate an accurate SQL query. You'll witness how enriching the prompt with descriptions of the relationships between tables can significantly influence and enhance the quality of the query produced by the model.

Can you notice the difference in the query?

  - Model: Anthropic Claude
  - Techniques:
    - Schema Incorporation
  - Use-Case: List all SF employees and their Manager's name
    - query1: With Schema Description
    - query2: With Schema Description with Tables Realationships


In [ ]:
question = "List the full names of all San Francisco office's employees, along with their managers full names"

prompt = """
You task is to generate the simplest and most effective SQL query.
Given these tables:
offices (
  officeCode,
  city,
  phone,
  addressLine1,
  addressLine2,
  state,
  country,
  postalCode,
  territory
);

employees (
  employeeNumber,
  lastName,
  firstName,
  extension,
  email,
  officeCode,
  reportsTo,
  jobTitle
);

Please be aware of tables columns, sqllite syntax, and the order of the operation in the SQL statement.
Please dont add any explanation or intros in your answer, the output should be only the SQL statement ready to run agaisnt the sqlite3 database
Please construct only the valid sqlite3 SQL statement to answer the following the question:
""" + question + """

sql
"""

In [ ]:
query_1_scenario_2 = invoke_anthropic_model(modelId, prompt)
print(query_1_scenario_2)

In [ ]:
question = "List the full names of all San Francisco office's employees, along with their managers full names"

prompt = """
You task is to generate the simplest and most effective SQL query.
Given these tables:
offices (
  officeCode,
  city,
  phone,
  addressLine1,
  addressLine2,
  state,
  country,
  postalCode,
  territory,
  PRIMARY KEY (officeCode)
);

employees (
  employeeNumber,
  lastName,
  firstName,
  extension,
  email,
  officeCode,
  reportsTo,
  jobTitle,
  PRIMARY KEY (employeeNumber),
  FOREIGN KEY (reportsTo) REFERENCES employees (employeeNumber),
  FOREIGN KEY (officeCode) REFERENCES offices (officeCode)
);

Please be aware of tables columns, sqllite syntax, and the order of the operation in the SQL statement.
Please dont add any explanation or intros in your answer, the output should be only the SQL statement ready to run agaisnt the sqlite3 database
Please construct only the valid sqlite3 SQL statement to answer the following the question:
""" + question + """

sql
"""

In [ ]:
query_2_scenario_2 = invoke_anthropic_model(modelId, prompt)
print(query_2_scenario_2)

# Queries Execution



In [ ]:
res = querySQL(query_1_scenario_2)
prettyJSON(res)

In [ ]:
res = querySQL(query_2_scenario_2)
prettyJSON(res)

In [ ]:
# Conclustion
# Result 1: Error - wrong column name
# Result 2: Works
# Result 3: Works

## Scenario  3 - 
In practical scenarios, the column names within databases often lack sufficient descriptiveness, making it challenging to comprehend their intended purpose or the nature of the data they represent

In this scenario, we have two databases: 'anycompany' and 'chaos_anycompany'. Some column names in the 'chaos_anycompany' database have been modified, resulting in a lack of descriptive information. We will explore how providing descriptions for the correct columns can guide the model in generating an accurate SQL query.

Bonus:
Try to get the same query after optimizing the prompt, e.g. remove irrelevnt columns

  - Model: Mistral Small
  - Techniques:
    - Schema Incorporation
  - Use-Case: Count customers that purchased motorcycles
    - query1: from anycompany, with Schema Description
    - query2: from chaos_anycompany with Schema Description
    - query3: from chaos_anycompany with Schema Description and relevant columns descripion 


In [ ]:
question = "Return the number of customers that purchased motorcycles"

prompt = """
You task is to generate the simplest and most effective SQL query.
Given these tables:

products (
  productCode,
  productName,
  productLine,
  productScale,
  productVendor,
  productDescription,
  quantityInStock,
  buyPrice,
  MSRP
);

customers (
  customerNumber,
  customerName,
  contactLastName,
  contactFirstName,
  phone,
  addressLine1,
  addressLine2,
  city,
  state,
  postalCode,
  country,
  salesRepEmployeeNumber,
  creditLimit
);

orders (
  orderNumber,
  orderDate,
  requiredDate,
  shippedDate,
  status,
  comments,
  customerNumber
);

orderdetails(
  orderNumber,
  productCode,
  quantityOrdered,
  priceEach,
  orderLineNumber 
);

Please be aware of tables columns, sqllite syntax!! and the order of the operation in the SQL statement.
Please dont add any explanation or intros in your answer, the output should be only the SQL statement ready to run agaisnt the sqlite3 database
Please construct only the valid sqlite3 SQL statement to answer the following the question:
""" + question + """

sql
"""

In [ ]:
query_1_scenario_3 = invoke_mistral_model(prompt, mistrasmall)
print(query_1_scenario_3)

In [ ]:
question = "Return the number of customers that purchased motorcycles"

prompt = """
You task is to generate the simplest and most effective SQL query.
Given these tables:

chaos_products (
  productCode,
  productName,
  pline,
  productScale,
  productVendor,
  productDescription,
  quantityInStock,
  buyPrice,
  MSRP
);

chaos_customers (
  customerNumber,
  customerName,
  contactLastName,
  contactFirstName,
  phone,
  addressLine1,
  addressLine2,
  city,
  state,
  postalCode,
  country,
  enumber,
  creditLimit
);

chaos_orders (
  orderNumber,
  orderDate,
  requiredDate,
  shippedDate,
  status,
  comments,
  cnumber
);

chaos_orderdetails(
  orderNumber,
  pcode,
  quantityOrdered,
  priceEach,
  orderLineNumber 
);

Please be aware of tables columns, sqllite syntax!! and the order of the operation in the SQL statement.
Please dont add any explanation or intros in your answer, the output should be only the SQL statement ready to run agaisnt the sqlite3 database
Please construct only the valid sqlite3 SQL statement to answer the following the question:
""" + question + """

sql
"""

In [ ]:
query_2_scenario_3 = invoke_mistral_model(prompt, mistrasmall)
print(query_2_scenario_3)

In [ ]:
question = "Return the number of customers that purchased motorcycles"

prompt = """
You task is to generate the simplest and most effective SQL query.
Given these tables:

chaos_products (
  productCode,
  productName,
  pline -- Product line such as Ships, Classic Cars, etc
  productScale,
  productVendor,
  productDescription,
  quantityInStock,
  buyPrice,
  MSRP
);

chaos_customers (
  customerNumber,
  customerName,
  contactLastName,
  contactFirstName,
  phone,
  addressLine1,
  addressLine2,
  city,
  state,
  postalCode,
  country,
  enumber,
  creditLimit
);

chaos_orders (
  orderNumber,
  orderDate,
  requiredDate,
  shippedDate,
  status,
  comments,
  cnumber
);

chaos_orderdetails(
  orderNumber,
  pcode,
  quantityOrdered,
  priceEach,
  orderLineNumber 
);

Please be aware of tables columns, sqllite syntax!! and the order of the operation in the SQL statement.
Please dont add any explanation or intros in your answer, the output should be only the SQL statement ready to run agaisnt the sqlite3 database
Please construct only the valid sqlite3 SQL statement to answer the following the question:
""" + question + """

sql
"""

In [ ]:
query_3_scenario_3 = invoke_mistral_model(prompt, mistrasmall)
print(query_3_scenario_3)

# Queries Execution



In [ ]:
res = querySQL(query_1_scenario_3)
prettyJSON(res)

In [ ]:
res = querySQL_chaos(query_2_scenario_3)
prettyJSON(res)

In [ ]:
res = querySQL_chaos(query_3_scenario_3)
prettyJSON(res)

In [ ]:
#query3 = "SELECT o.orderNumber AS order_id, o.orderDate AS order_date, o.status AS order_status, COUNT(od.productCode) AS total_products FROM orders o INNER JOIN orderdetails od ON o.orderNumber = od.orderNumber LEFT JOIN products p ON od.productCode = p.productCode WHERE o.status != 'Cancelled' GROUP BY o.orderNumber, o.orderDate, o.status"
#res = querySQL(query3)
#prettyJSON(res)

## Scenario  4
This scenario showcases how a smaller model fails to provide an accurate SQL query for a given question, while a larger model succeeds in generating the correct query.

  - Models: 
    - Mistral AI - Mistral Small
    - Anthropic Claude
  - Use Case: List all customers who have ordered products from all product lines
    - qurey1: With Mistral Small
    - query2: With Anthropic Claude
  
Bonus: Try enhancing the prompt to help Mistral Small provide an accurate query


In [ ]:
TIP: Few Shots

In [ ]:
question = "List customer names who have ordered products from all product lines."

prompt = """
You task is to generate the simplest and most effective SQL query. Output only SQL, without comments.
Given these tables:
productlines -- This table consist of all product lines
(
  productLine varchar(50),
  textDescription varchar(4000) DEFAULT NULL,
  htmlDescription mediumtext,
  image mediumblob,
  PRIMARY KEY (productLine)
);

products (
  productCode varchar(15),
  productName varchar(70) NOT NULL,
  productLine varchar(50) NOT NULL -- product line such as Ships, Classic Cars, etc,
  productScale varchar(10) NOT NULL,
  productVendor varchar(50) NOT NULL,
  productDescription text NOT NULL,
  quantityInStock smallint(6) NOT NULL,
  buyPrice decimal(10,2) NOT NULL,
  MSRP decimal(10,2) NOT NULL,
  PRIMARY KEY (productCode),
  FOREIGN KEY (productLine) REFERENCES productlines (productLine)
);

customers (
  customerNumber int,
  customerName varchar(50) NOT NULL,
  contactLastName varchar(50) NOT NULL,
  contactFirstName varchar(50) NOT NULL,
  phone varchar(50) NOT NULL,
  addressLine1 varchar(50) NOT NULL,
  addressLine2 varchar(50) DEFAULT NULL,
  city varchar(50) NOT NULL,
  state varchar(50) DEFAULT NULL,
  postalCode varchar(15) DEFAULT NULL,
  country varchar(50) NOT NULL,
  salesRepEmployeeNumber int DEFAULT NULL,
  creditLimit decimal(10,2) DEFAULT NULL,
  PRIMARY KEY (customerNumber),
  FOREIGN KEY (salesRepEmployeeNumber) REFERENCES employees (employeeNumber)
);

orders (
  orderNumber int,
  orderDate date NOT NULL,
  requiredDate date NOT NULL,
  shippedDate date DEFAULT NULL,
  status varchar(15) NOT NULL,
  comments text,
  customerNumber int NOT NULL,
  PRIMARY KEY (orderNumber),
  FOREIGN KEY (customerNumber) REFERENCES customers (customerNumber)
);

orderdetails (
  orderNumber int,
  productCode varchar(15) NOT NULL,
  quantityOrdered int NOT NULL,
  priceEach decimal(10,2) NOT NULL,
  orderLineNumber smallint(6) NOT NULL,
  PRIMARY KEY (orderNumber,productCode),
  FOREIGN KEY (orderNumber) REFERENCES orders (orderNumber),
  FOREIGN KEY (productCode) REFERENCES products (productCode)
);

Please be aware of tables,columns, and primary and foreign key.
Please dont add any explanation or intros in your answer, the output should be only the SQL statement ready to run agaisnt the sqlite3 database
Please construct only the valid sqlite3 SQL statement and syntax to answer the following the question:
""" + question + """

sql
"""

In [ ]:
query_1_scenario_4 = invoke_mistral_model(prompt, mistrasmall)
print(query_1_scenario_4)

In [ ]:
query_2_scenario_4 = invoke_anthropic_model(modelId, prompt)
print(query_2_scenario_4)

In [ ]:
res = querySQL(query_1_scenario_4)
prettyJSON(res)

In [ ]:
res = querySQL( query_2_scenario_4 )
prettyJSON(res)

## Scenario  5
We'll use a Single Document RAG model with the database schema to help generate SQL queries like previous scenarios. The RAG will provide relevant schema details, which we'll include in the prompt for our main model. This technique is useful for databases with many tables and columns

This technique is particularly beneficial when dealing with databases comprising numerous tables and columns, as it streamlines the process of obtaining and integrating the relevant schema details into the prompt, thereby enhancing the model's ability to generate accurate SQL queries

Here we also use a tempate for the prompt

  - Method: Single Document Retrieval Augmented Generation (RAG) with Bedrock Knowledge Bases
  - Model: Anthropic Claude 3 Sonnet
  - Reference: https://aws.amazon.com/blogs/machine-learning/knowledge-bases-in-amazon-bedrock-now-simplifies-asking-questions-on-a-single-document/



In [ ]:
question1 = "List customer names who have ordered products from all product lines"

In [ ]:
question2 = "Return the number of customers that purchased motorcycles"

In [ ]:
question3 = "List the full names of all San Francisco office's employees, along with their managers full names"

In [ ]:
question_for_rag_template = "Provide schema representation containing table name, column names, and relationships between tables for the following question: {0}  The output should be a json containing the table's description. Don't write any explanations and comments"

In [ ]:
question1_for_rag = question_for_rag_template.format(question1)

In [ ]:
question2_for_rag = question_for_rag_template.format(question2)

In [ ]:
question3_for_rag = question_for_rag_template.format(question3)

In [ ]:
res1 = retrieveAndGenerate(question1_for_rag)
prettyJSON(res1)

In [ ]:
res2 = retrieveAndGenerate(question2_for_rag)
prettyJSON(res2)

In [ ]:
res3 = retrieveAndGenerate(question3_for_rag)
prettyJSON(res3)

In [77]:
prompt1 = """
You task is to generate the simplest and most effective SQL query. Output only SQL, without comments.
Given these tables:
    """ + res1 + """

Please be aware of tables,columns, and primary and foreign key.
Dont add any explanation or intros in your answer, the output should be only the SQL statement ready to run agaisnt the sqlite3 database
Construct only the valid sqlite3 SQL statement and syntax to answer the following the question:
""" + question1 + """

sql
"""

prompt2 = """
You task is to generate the simplest and most effective SQL query. Output only SQL, without comments.
Given these tables:
    """ + res2 + """

Please be aware of tables,columns, and primary and foreign key.
Dont add any explanation or intros in your answer, the output should be only the SQL statement ready to run agaisnt the sqlite3 database
Construct only the valid sqlite3 SQL statement and syntax to answer the following the question:
""" + question2 + """

sql
"""

prompt3 = """
You task is to generate the simplest and most effective SQL query. Output only SQL, without comments.
Given these tables:
    """ + res3 + """

Please be aware of tables,columns, and primary and foreign key.
Dont add any explanation or intros in your answer, the output should be only the SQL statement ready to run agaisnt the sqlite3 database
Construct only the valid sqlite3 SQL statement and syntax to answer the following the question:
""" + question3 + """

sql
"""

In [59]:
query_1_scenario_5 = invoke_anthropic_model(modelId, prompt1)
print(query_1_scenario_5)

In [60]:
query_2_scenario_5 = invoke_anthropic_model(modelId, prompt2)
print(query_2_scenario_1)

In [78]:
query_3_scenario_5 = invoke_anthropic_model(modelId, prompt3)
print(query_3_scenario_5)

In [101]:
res = querySQL(query_1_scenario_5)
prettyJSON(res)

In [102]:
res = querySQL(query_2_scenario_1)
prettyJSON(res)

In [103]:
res = querySQL(query_3_scenario_5)
prettyJSON(res)

In [ ]:
res1 = retrieveAndGenerate(question_for_rag)
prettyJSON(res1)

## Test your knowledge 
Apply the learned techniques to write multiple prompts for the following questions, using any of the workshop models:

 - What are the names of all the product lines?
 - List all the products with their names, scales, and vendors.
 - Show the first and last names of all employe
 - Find the names and credit limits of customers who have a sales representative assigned.
 - Retrieve the order numbers, order dates, and statuses for orders placed in the year 2022.
 - Get the product codes and quantities ordered for a specific order number.
 - Calculate the total revenue (quantity ordered * price each) for each product line, ordered by revenue in descending order.
 - Retrieve the names of customers who have placed orders for products from all product lines, along with the number of distinct product lines they have ordered from.


In [22]:
question = ""

prompt = """
You task is to generate the simplest and most effective SQL query.
Given these tables:



Please be aware of tables columns, sqllite syntax, and the order of the operation in the SQL statement.
Please dont add any explanation or intros in your answer, the output should be only the SQL statement ready to run agaisnt the sqlite3 database
Please construct only the valid sqlite3 SQL statement to answer the following the question:
""" + question + """

sql
"""

In [ ]:
query_rag = question_for_rag_template.format(question)

In [ ]:
res = retrieveAndGenerate(query_rag)
prettyJSON(res)

In [ ]:
query_1 = invoke_mistral_model(prompt, mistral7b)
print(query_1)

In [ ]:
query_2 = invoke_anthropic_model(modelId, prompt)
print(query_2)

In [ ]:
res = querySQL(query_1)
prettyJSON(res)

In [ ]:
res = querySQL(query_2)
prettyJSON(res)